In [1]:
# using Pkg; Pkg.add("BenchmarkTools")

In [2]:
function fib(n)
    n <= 2 ? 1 : fib(n-1) + fib(n-2)
end

fib (generic function with 1 method)

In [3]:
fib(4)

3

In [4]:
@time fib(40)

  0.298376 seconds (5 allocations: 176 bytes)


102334155

In [5]:
function memoit(f::Function, p)
    if !isdefined(Main, :memoit_cache)
        global memoit_cache = Dict{Function,Dict{Any,Any}}()
    end
    c = haskey(memoit_cache, f) ? memoit_cache[f] : memoit_cache[f]=Dict()
    haskey(c, p) ? c[p] : c[p] = f(p)
end

memoit (generic function with 1 method)

In [6]:
function fib2(n)
    n <= 2 ? 1 : memoit(fib2, n-1) + memoit(fib2, n-2)
end

fib2 (generic function with 1 method)

In [7]:
fib2(4)

3

In [8]:
@time fib2(40)

  0.000092 seconds (65 allocations: 2.703 KiB)


102334155

In [10]:
macro memo(e)
    println("macro @memo is run: ", e, " ", e.args)
    (!(typeof(e) <: Expr) || !(e.head == :call)) &&
        error("wrong @memo params")
    return quote                     # 1 引数の関数だけを扱う
        memoit($(esc(e.args[1])), $(esc(e.args[2])))
    end
end

@memo (macro with 1 method)

In [11]:
function fib3(n)
    n <= 2 ? 1 : (@memo fib3(n-1)) + (@memo fib3(n-2))
end

macro @memo is run: fib3(n - 1) Any[:fib3, :(n - 1)]
macro @memo is run: fib3(n - 2) Any[:fib3, :(n - 2)]


fib3 (generic function with 1 method)

In [12]:
fib3(4)

3

In [13]:
@time fib3(40)

  0.000045 seconds (58 allocations: 2.328 KiB)


102334155

#### @generatedマクロを用いてループアンローリングする


In [14]:
function sumx1(objs...)
    isempty(objs) && return 0
    total = objs[1].x
    for i in 2:length(objs)
        total += objs[i].x
    end
    total
end

sumx1 (generic function with 1 method)

In [15]:
struct A x::Int end
struct B x::Float64 end

In [16]:
sumx1(A(5), B(7))

12.0

In [17]:
sumx1(A(5), A(17))

22

In [19]:
@generated function sumx2(objs...)
    isempty(objs) && return 0    #  引数がなかった場合のデフォルト返り値
    total = :(objs[1].x)
    for i in 2:length(objs)
        total = :($total + objs[$i].x)
    end
    total 
end

sumx2 (generic function with 1 method)

In [20]:
sumx2(A(5), B(7)) == sumx1(A(5), B(7))

true

In [21]:
sumx2(A(5), A(17)) == sumx1(A(5), A(17))

true

In [22]:
using BenchmarkTools
const valsx = ([A(i) for i=1:10]..., [B(i) for i=1:10]...)
typeof(valsx)

Tuple{A,A,A,A,A,A,A,A,A,A,B,B,B,B,B,B,B,B,B,B}

In [23]:
@btime sumx1(valsx...)

  532.419 ns (38 allocations: 3.56 KiB)


110.0

In [24]:
@btime sumx2(valsx...)

  0.026 ns (0 allocations: 0 bytes)


110.0

### How it works...

In [23]:
macro example(v)
    :(($v, $(esc(v))))
end

@example (macro with 1 method)

In [24]:
function f() 
    x = 1
    @example x
end

f (generic function with 1 method)

In [25]:
x = 10

10

In [26]:
f()

(10, 1)

In [27]:
@macroexpand @example x

:((Main.x, x))

In [28]:
n=5

5

In [29]:
macroexpand(Main, :(@memo fib3(n-1)))

macro @memo is run: fib3(n - 1) Any[:fib3, :(n - 1)]


quote
    #= In[9]:6 =#
    Main.memoit(fib3, n - 1)
end

### 説明しよう

In [25]:
@code_lowered sumx2(A(1), B(2))

CodeInfo(
    @ In[19]:2 within `sumx2'
   ┌ @ In[19]:2 within `macro expansion'
1 ─│ %1 = Base.getindex(objs, 1)
│  │ %2 = Base.getproperty(%1, :x)
│  │ %3 = Base.getindex(objs, 2)
│  │ %4 = Base.getproperty(%3, :x)
│  │ %5 = %2 + %4
└──│      return %5
   └
)

In [26]:
@code_lowered sumx1(A(1), B(2))

CodeInfo(
1 ─       Core.NewvarNode(:(total))
│         Core.NewvarNode(:(@_4))
│   %3  = Main.isempty(objs)
└──       goto #4 if not %3
2 ─       return 0
3 ─       goto #5
4 ─       false
5 ┄ %8  = Base.getindex(objs, 1)
│         total = Base.getproperty(%8, :x)
│   %10 = Main.length(objs)
│   %11 = 2:%10
│         @_4 = Base.iterate(%11)
│   %13 = @_4 === nothing
│   %14 = Base.not_int(%13)
└──       goto #8 if not %14
6 ┄ %16 = @_4
│         i = Core.getfield(%16, 1)
│   %18 = Core.getfield(%16, 2)
│   %19 = total
│   %20 = Base.getindex(objs, i)
│   %21 = Base.getproperty(%20, :x)
│         total = %19 + %21
│         @_4 = Base.iterate(%11, %18)
│   %24 = @_4 === nothing
│   %25 = Base.not_int(%24)
└──       goto #8 if not %25
7 ─       goto #6
8 ┄       return total
)

In [27]:
@code_warntype sumx2(A(1), B(2))

Variables
  #self#::Core.Compiler.Const(sumx2, false)
  objs::Tuple{A,B}

Body::Float64
1 ─ %1 = Base.getindex(objs, 1)::A
│   %2 = Base.getproperty(%1, :x)::Int64
│   %3 = Base.getindex(objs, 2)::B
│   %4 = Base.getproperty(%3, :x)::Float64
│   %5 = (%2 + %4)::Float64
└──      return %5


In [28]:
@code_warntype sumx1(A(1), B(2))

Variables
  #self#::Core.Compiler.Const(sumx1, false)
  objs::Tuple{A,B}
  total::Union{Float64, Int64}
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─       Core.NewvarNode(:(total))
│         Core.NewvarNode(:(@_4))
│   %3  = Main.isempty(objs)::Core.Compiler.Const(false, false)
└──       goto #3 if not %3
2 ─       Core.Compiler.Const(:(return 0), false)
└──       Core.Compiler.Const(:(goto %8), false)
3 ┄       false
│   %8  = Base.getindex(objs, 1)::A
│         (total = Base.getproperty(%8, :x))
│   %10 = Main.length(objs)::Core.Compiler.Const(2, false)
│   %11 = (2:%10)::Core.Compiler.Const(2:2, false)
│         (@_4 = Base.iterate(%11))
│   %13 = (@_4::Core.Compiler.Const((2, 2), false) === nothing)::Core.Compiler.Const(false, false)
│   %14 = Base.not_int(%13)::Core.Compiler.Const(true, false)
└──       goto #6 if not %14
4 ─ %16 = @_4::Core.Compiler.Const((2, 2), false)::Core.Compiler.Const((2, 2), false)
│         (i = Core.getfield(%16, 1))
│   %18 = 

In [29]:
@code_warntype sumx2(A(1), B(1), B(2))

Variables
  #self#::Core.Compiler.Const(sumx2, false)
  objs::Tuple{A,B,B}

Body::Float64
1 ─ %1 = Base.getindex(objs, 1)::A
│   %2 = Base.getproperty(%1, :x)::Int64
│   %3 = Base.getindex(objs, 2)::B
│   %4 = Base.getproperty(%3, :x)::Float64
│   %5 = (%2 + %4)::Float64
│   %6 = Base.getindex(objs, 3)::B
│   %7 = Base.getproperty(%6, :x)::Float64
│   %8 = (%5 + %7)::Float64
└──      return %8
